In [2]:
import pm4py, os

#import event logs
if __name__ == "__main__":
    log = pm4py.read_xes('BPI_Challenge_2019-3-w-after.xes')

parsing log, completed traces ::   0%|          | 0/15182 [00:00<?, ?it/s]

In [32]:
# exclude all traces that are not between 2018-01-0 and 2019-01-28
def filter_timeframe(log):
    return pm4py.filter_time_range(log, "2018-01-01 00:00:00", "2019-01-28 23:59:59", mode='traces_contained')

# group the eventlog by the values of the documentType attribute
def group_by_documentType(log):
    log_ecPurchaseOrder = pm4py.filter_trace_attribute_values(log, "(case) Document Type", ["EC Purchase order"], retain = True)
    log_frameworkOrder = pm4py.filter_trace_attribute_values(log, "(case) Document Type", ["EC Purchase order"], retain = True)
    log_standardPO = pm4py.filter_trace_attribute_values(log, "(case) Document Type", ["EC Purchase order"], retain = True)
    return log_ecPurchaseOrder, log_frameworkOrder, log_standardPO


# removes traces, that have not one of the specified start and end-activities
def remove_incomplete_cases(log_ecPurchaseOrder, log_frameworkOrder, log_standardPO):
    
    # EC Purchase order
    log_ecPurchaseOrder = pm4py.filter_start_activities(log_ecPurchaseOrder, ["Create Purchase Order Item",
                    "Create Purchase Requisition Item", "SRM: Created", "Vendor creates invoice"])
    log_ecPurchaseOrder = pm4py.filter_end_activities(log_ecPurchaseOrder, ["Clear Invoice", "Delete Purchase Order Item"])
    
    # Framework order
    log_frameworkOrder = pm4py.filter_start_activities(log_frameworkOrder, ["Create Purchase Order Item",
                    "Create Purchase Requisition Item", "Vendor creates invoice"])
    log_frameworkOrder = pm4py.filter_end_activities(log_ecPurchaseOrder, ["Clear Invoice", "Delete Purchase Order Item"])
    
    # Standard PO
    log_standardPO = pm4py.filter_start_activities(log_standardPO, ["Create Purchase Order Item",
                    "Create Purchase Requisition Item", "Vendor creates invoice"])
    log_standardPO = pm4py.filter_end_activities(log_ecPurchaseOrder, ["Clear Invoice", "Delete Purchase Order Item"])
    
    return log_ecPurchaseOrder, log_frameworkOrder, log_standardPO

def filter_out_infrequent_variants(log_ecPurchaseOrder, log_frameworkOrder, log_standardPO):
    
    # EC Purchase order
    log_ecPurchaseOrder = pm4py.filter_variants_by_coverage_percentage(log_ecPurchaseOrder, 0.0002)
    
    # Framework order
    log_frameworkOrder = pm4py.filter_variants_by_coverage_percentage(log_ecPurchaseOrder, 0.01)
    
    # Standard PO
    log_standardPO = pm4py.filter_variants_by_coverage_percentage(log_ecPurchaseOrder, 0.0002)
    
    return log_ecPurchaseOrder, log_frameworkOrder, log_standardPO

# ------ NOT ACTIVE ------
#filter out variants, that only consist of events:
#     "Create Purchase Order Item", "Delete Purchase Order Item"
def filter_out_cpoi_dpoi(log):
    return pm4py.filter_variants(log, [["Create Purchase Order Item", "Delete Purchase Order Item"]], retain=False)

#write event log to a xes file in the preprocessed directory
def write_log_to_file(log, filename):
    path = os.path.join(os.path.dirname(os.getcwd()), 'Preprocessed', filename)
    pm4py.write_xes(log, path)

In [35]:
# Preprocessing pipeline

# 1. Filter on timeframe
filtered_log = filter_timeframe(log)

# 2. Group by documentType
log_ecPurchaseOrder, log_frameworkOrder, log_standardPO = group_by_documentType(filtered_log)

# 3. Remove incomplete cases
log_ecPurchaseOrder, log_frameworkOrder, log_standardPO = remove_incomplete_cases(log_ecPurchaseOrder, log_frameworkOrder, log_standardPO)

# 4. Filter out infrequent variants
log_ecPurchaseOrder, log_frameworkOrder, log_standardPO = filter_out_infrequent_variants(log_ecPurchaseOrder, log_frameworkOrder, log_standardPO)

# write filtered logs to files
write_log_to_file(log_ecPurchaseOrder, 'EC Purchase order_preprocessed.xes')
write_log_to_file(log_frameworkOrder, 'Framework order_preprocessed.xes')
write_log_to_file(log_standardPO, 'Standard PO_preprocessed.xes')

exporting log, completed traces ::   0%|          | 0/346 [00:00<?, ?it/s]

exporting log, completed traces ::   0%|          | 0/217 [00:00<?, ?it/s]

exporting log, completed traces ::   0%|          | 0/346 [00:00<?, ?it/s]